# Vispy's MindMap API: a first implementation to identify appropriate design requirements

Most of the work here will be reusable for other type of graph but primary effort is devoted to implement a working and esasy to use API for mindMaps.

This work is devoted to identify the requirements of such an API from the users point of view and to select a good design and a good implementation strategy.


## Scope

Let's start with a very limited scope:

  - Targeted users want to draw a mindMap.
  - The user already have a mindMap in the form of a networkx object or of a markdown file. 
  - The graph to plot is not a directed graph.
  - A typical workflow of the API would be:  
    - input a graph as a whole or from subgraphs,
    - edit the rendering styles and layout,
    - interactivelly adjust the styles and layout
    - export the rendering parameters for reuse purpose
    - export the graphData (connectivity and attributes)
    - save the graph image.

## Requirements

### GraphData input

  - GraphData is any data that can be represented as a graph.
  - Commond mindMap have less than 100 nodes.
  - The user should not have to preprocess his graphData.
  - Standard input format should be recognized: let start with networkx graph object, numpy adjency matrix and scipy sparse adjency matrix.
  - Non standard but common _things_ that can be visualized as a graph should be recognize: let start with a markdown file.
  - It should be possible to pass the graphData as a subgraph to compose a graph.

###  Rendering edition: styles

   - The API must have default rendering parameters that generate beautiful rendering most of the time.
   - Nodes and edges style paramters should be changed i) for all, ii) by groups or iii) individually.
   - Other elements like background, boxed, titles, axes, 

###  Rendering edition: layout

   - User should note have to set the node coordinate.
   - User should be able to select the layout method.
   - User should be able to set different layout for different subgraphs
   - The layout parameters should take into account the space occupyed by the text in each node (remember that we want to have a mindMap API...)


### Interactive adjustment of the rendering


### Saving the rendering parameters

### Export the graphData

### Save the graph image


## Design 








#API's example of usage

### Starting from a networkx graph object

Generate a graph:

    from networkx import nx
   
    head="collection"
    sac=["numpy","gloo","fetchcode","BaseCollection","ModularProgram","EventEmitter"]

    G=nx.Graph()
    for node in sac:
        G.add_edge(head,node)

### Input the graphData

    from vispy import mindmap
    
    map=mindmap() 
    map.graphData(type="networkx", source=G)  # import a networkx graph
    
### Visualize the graph

    map.show()



In [1]:
from vispy import gloo
from vispy import app
import numpy as np

class graphCanvas(app.Canvas):
    

    def __init__(self):
        app.Canvas.__init__(self, keys='interactive',size=(1600,900))
        self.ps = self.pixel_scale
        self.node_vert_shader="""
attribute vec2  a_position;
attribute vec3  a_color;
attribute float a_size;
varying vec4 v_fg_color;
varying vec4 v_bg_color;
varying float v_radius;
varying float v_linewidth;
varying float v_antialias;
void main (void) {
    v_radius = a_size;
    v_linewidth = 1.0;
    v_antialias = 1.0;
    v_fg_color  = vec4(0.0,0.0,0.0,0.5);
    v_bg_color  = vec4(a_color,    1.0);
    gl_Position = vec4(a_position, 0.0, 1.0);
    gl_PointSize = 2.0*(v_radius + v_linewidth + 1.5*v_antialias);
}
"""
        self.node_frag_shader="""
#version 120
varying vec4 v_fg_color;
varying vec4 v_bg_color;
varying float v_radius;
varying float v_linewidth;
varying float v_antialias;
void main()
{
    float size = 2.0*(v_radius + v_linewidth + 1.5*v_antialias);
    float t = v_linewidth/2.0-v_antialias;
    float r = length((gl_PointCoord.xy - vec2(0.5,0.5))*size);
    float d = abs(r - v_radius) - t;
    if( d < 0.0 )
        gl_FragColor = v_fg_color;
    else
    {
        float alpha = d/v_antialias;
        alpha = exp(-alpha*alpha);
        if (r > v_radius)
            gl_FragColor = vec4(v_fg_color.rgb, alpha*v_fg_color.a);
        else
            gl_FragColor = mix(v_bg_color, v_fg_color, alpha);
    }
}
"""

    def input_graphData(self,connectivity):
        # Create vertices
        self.n=len(connectivity)
        
        self.connectivity=connectivity


    def set_nodeStyle(self):   
        # put anything for now...
        self.v_color = np.random.uniform(0, 1, (self.n, 3)).astype(np.float32)
        self.v_size = np.random.uniform(2*self.ps, 12*self.ps, (self.n, 1)).astype(np.float32)
        
    def aestheticPleasing_layout(self):
         # should not be hard coded for each method ... but let start with this.
            
        self.v_position=initAesthetic(self.connectivity).astype(np.float32)
        
    def on_timer(self, event):
 
        self.pos=updateAesthetic(self.v_position,self.connectivity).astype(np.float32)
        self.program['a_position'] = self.v_position
        self.update()

        


        

    def compile(self):
        # pretreatment should be finished. Add some tests to check if its correctly done...
        self.set_nodeStyle()
        
        
        self.program = gloo.Program(self.node_vert_shader, self.node_frag_shader)
        self.program['a_color'] = gloo.VertexBuffer(self.v_color)
        self.program['a_position'] = gloo.VertexBuffer(self.v_position)
        self.program['a_size'] = gloo.VertexBuffer(self.v_size)
        gloo.set_state(clear_color='white', blend=True,
                       blend_func=('src_alpha', 'one_minus_src_alpha'))
        self._timer = app.Timer('auto', connect=self.on_timer, start=True)

    def on_resize(self, event):
        gloo.set_viewport(0, 0, *event.physical_size)

    def on_draw(self, event):
        gloo.clear(color=True, depth=True)
        self.program.draw('points')
        
        
    




class mindMap(object):
    """
# API scope

  - Draw a mindMap 
    
    
# Requirements
  - import data from networkx object.
  - draw the mindMap
    
# Design
    
# Specifications    
    
    """
    def __init__(self):
        
        self.canvas=graphCanvas()
        self.nodeStyle=nodeStyle() # not in use yet...
        
  
        
    def graphData(self,data,source="networkx"):   
        """
        Data loader from standard format or objects.
        
        Where do I put the data?
        """  
        
        try:
            self.connectivity,self.nodesAttributes,self.edgesAttributes=eval("loadGraph_"+source)(data)
        except:
            print("Loading data from {} source is not supported.".format(source))
    
    def idraw(self):

        self.canvas.input_graphData(self.connectivity)
        self.canvas.aestheticPleasing_layout()
        self.canvas.compile()
        self.canvas.show()



        
    
    
    
    
def loadGraph_networkx(G):
    # utility to load networkx and test the input graph
    try:
        import networkx as nx
    except:
        print("Unable to load networkx library.")
        raise
    assert G.__class__ is nx.classes.graph.Graph, "Only networkx graph is supported by networkxParser."
            
    # transfer networkx's connectivity and graph attributes
        
    n=G.number_of_nodes()
        
    connectivity=np.zeros((n,n))
    nodesAttributes={}
    edgesAttributes={}
        
    idx=0
    for node,attr in G.nodes(data=True):  
        if len(attr)==0:
            t={"idx":idx}        
        else:
            t=attr.update({"idx":idx})
       # print(idx)
        nodesAttributes[node]=t
        idx+=1
    
    ldx=0
    for i,j,attr in G.edges(data=True):
      #  print(i,j,nodesAttributes[i])
        idx=nodesAttributes[i]["idx"]
        jdx=nodesAttributes[j]["idx"]
     #   print(idx,jdx)
        connectivity[idx,jdx]+=1
        edgesAttributes[ldx]=attr.update({"idx":(idx,jdx)})  
        ldx+=1

        
   # print(connectivity)
        
    return connectivity,nodesAttributes,edgesAttributes
        
    
        
            
        

In [2]:
# see aesthetically pleasing method in http://arxiv.org/pdf/1201.3011.pdf
  

def initAesthetic(connectivity):
    # set random position
    n= len(connectivity)
    pos=np.zeros((n,2))
    
    for i in np.arange(n-1):
        pos[i][0]=np.random.rand()
        pos[i][1]=np.random.rand()
        
    
    
    return pos   

def updateAesthetic(pos,connectivity):
  #set constants
    c1=1.1
    c2=.4
    c3=.2
    c4=0.05
    n= len(pos)
    
    
    

    
    # force on each vertex + deplacement
    
    
    for i in np.arange(n):
 
        cft=0
     #   print(np.arange(i+1,n-1))
        for j in np.arange(i+1,n):

            cf=0
      #      print(i,j)
            r=pos[i]-pos[j]
            d=distance(pos[i],pos[j])

        #    print("p1 {}  et p2 {}".format(pos[i],pos[j]))
        #    print("unitVector {}".format(e))
            
            if not connectivity[j,i]>10e-10:
                # with edge => attraction
                cf= -c1*np.log(d/c2)
       #         print("attraction: {},{}".format(i,j))  
            else:
                # without edge => repulsion
                cf=c3/np.sqrt(d)
        #        print("repulsion: {},{}".format(i,j))
            cft+=cft+cf*r

            #print("")
       # print(cf)
        pos[i]+=c4*cft
   # print(pos)
    
            
    
    return pos
    
def distance(p1,p2):
    # assume numpy 2D arrays (should be replace by vispy in-built vector transformations)
    return np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
def unitVector(p1,p2):
    # assume numpy 2D arrays (should be replace by vispy in-built vector transformations)
    n=p1-p2
    norme=np.sqrt(sum(n**2))
    return n/norme

In [3]:
# Graph dataset



class graphDataset():
    """
    Store and handle graph data.
    """
    def __init__(self):
        self.nodes=None
        self.edges=None
        self.nodesAttributes=None
        self.edgesAttributes=None

        
# Style parameters        
        
class nodeStyle():
    
    def __init__(self,color=(0,0,153),size=15):
        self.color=color
        self.size=size
        
       
 



In [4]:
import networkx as nx

center="collection"
sac=["numpy","gloo","fetchcode","BaseCollection","ModularProgram","EventEmitter"]


G=nx.Graph()

for node in sac:
    G.add_edge(center,node)

m=mindMap()
m.graphData(G)


INFO: Could not import backend "PyQt4":
No module named 'PyQt4'
INFO:vispy:Could not import backend "PyQt4":
No module named 'PyQt4'
INFO: Could not import backend "PyQt5":
cannot import name 'QtOpenGL'
INFO:vispy:Could not import backend "PyQt5":
cannot import name 'QtOpenGL'
INFO: Could not import backend "PySide":
dlopen(/opt/local/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/PySide/QtOpenGL.so, 2): Library not loaded: libpyside.cpython-34m.1.2.dylib
  Referenced from: /opt/local/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/PySide/QtOpenGL.so
  Reason: image not found
INFO:vispy:Could not import backend "PySide":
dlopen(/opt/local/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/PySide/QtOpenGL.so, 2): Library not loaded: libpyside.cpython-34m.1.2.dylib
  Referenced from: /opt/local/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/PySide/QtOpenGL.so
  Reason: image

In [ ]:
m.idraw()
app.run()